# DeepMatch 样例代码
- https://github.com/shenweichen/DeepMatch
- https://deepmatch.readthedocs.io/en/latest/

# 下载movielens-1M数据 安装依赖包

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip 
! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py
! unzip -o ml-1m.zip 
! pip install -q deepmatch

--2021-07-04 12:57:19--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  27.8MB/s    in 0.2s    

2021-07-04 12:57:19 (27.8 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2021-07-04 12:57:19--  https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5642 (5.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]  

# 导入需要的库

In [2]:
! pip install tensorflow-gpu
! pip install tensorflow


     |████████████████████████████████| 454.3MB 38kB/s 


In [3]:
import pandas as pd

from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss


DeepCTR version 0.8.6 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.6


# 读取数据

In [4]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames)

data = pd.merge(pd.merge(ratings,movies),user)#.iloc[:10000]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


# 构建特征列，训练模型，导出embedding

In [5]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [6]:
#data = pd.read_csvdata = pd.read_csv("./movielens_sample.txt")
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", ]
SEQ_LEN = 50
negsample = 0

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']
feature_max_idx = {}
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1

In [7]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1105,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1,1,11,1589
1,1,640,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,1,1,11,1589
2,1,854,3,978301968,My Fair Lady (1964),Musical|Romance,1,1,11,1589
3,1,3178,4,978300275,Erin Brockovich (2000),Drama,1,1,11,1589
4,1,2163,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,1,11,1589


In [8]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')

item_profile = data[["movie_id"]].drop_duplicates('movie_id')

In [9]:
user_profile.set_index("user_id", inplace=True)

In [10]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1105,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1,1,11,1589
1,1,640,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,1,1,11,1589
2,1,854,3,978301968,My Fair Lady (1964),Musical|Romance,1,1,11,1589
3,1,3178,4,978300275,Erin Brockovich (2000),Drama,1,1,11,1589
4,1,2163,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,1,11,1589


In [11]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)

In [12]:
user_item_list[:10]

user_id
1     [1105, 640, 854, 3178, 2163, 1108, 1196, 2600,...
2     [1105, 2890, 2129, 1783, 1118, 1849, 1155, 126...
3     [2163, 1108, 1179, 1782, 254, 2899, 628, 1121,...
4     [1026, 2489, 254, 1849, 3236, 1121, 467, 1107,...
5     [3178, 2163, 859, 2890, 1575, 2558, 145, 2489,...
6     [854, 3178, 582, 971, 2129, 48, 1440, 575, 1, ...
7     [1179, 1849, 628, 2709, 3342, 2892, 1479, 3033...
8     [2890, 514, 1575, 2970, 1659, 145, 1, 2489, 18...
9     [3178, 2163, 690, 514, 1575, 709, 2103, 2558, ...
10    [854, 3178, 2163, 1108, 1196, 2600, 581, 859, ...
Name: movie_id, dtype: object

In [13]:
for idx, (reviewerID, hist) in enumerate(data.groupby('user_id')):
  if idx > 10:
    break
  print(hist)

    user_id  movie_id  rating  timestamp  ... gender age  occupation   zip
0         1      1105       5  978300760  ...      1   1          11  1589
1         1       640       3  978302109  ...      1   1          11  1589
2         1       854       3  978301968  ...      1   1          11  1589
3         1      3178       4  978300275  ...      1   1          11  1589
4         1      2163       5  978824291  ...      1   1          11  1589
5         1      1108       3  978302268  ...      1   1          11  1589
6         1      1196       5  978302039  ...      1   1          11  1589
7         1      2600       5  978300719  ...      1   1          11  1589
8         1       581       4  978302268  ...      1   1          11  1589
9         1       859       4  978301368  ...      1   1          11  1589
10        1       582       5  978824268  ...      1   1          11  1589
11        1       878       4  978301752  ...      1   1          11  1589
12        1      2206    

In [14]:
train_set, test_set = gen_data_set(data, negsample)

train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)  #here equals to read data from csv directly
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

100%|██████████| 6040/6040 [00:10<00:00, 575.25it/s] 


6 6


In [15]:
train_model_input

{'age': array([5, 3, 3, ..., 4, 3, 6]),
 'gender': array([1, 1, 2, ..., 2, 2, 2]),
 'hist_len': array([ 91, 287, 277, ...,  47, 449,  25]),
 'hist_movie_id': array([[1171, 1032, 1770, ..., 1128, 1762, 1163],
        [1048, 1937,  964, ...,  994, 1092,   47],
        [  93, 1598, 2219, ...,  167, 1196, 1834],
        ...,
        [1866, 1540, 2427, ...,    0,    0,    0],
        [1215,   21,  348, ..., 1066,  576, 1652],
        [ 175, 1706,   21, ...,    0,    0,    0]], dtype=int32),
 'movie_id': array([1159, 2721,  155, ..., 1446,  538, 2309]),
 'occupation': array([ 3,  5,  5, ...,  1,  3, 13]),
 'user_id': array([1140, 4771, 4028, ..., 5523, 3683, 2174]),
 'zip': array([2295,  227,  106, ..., 2037, 1109, 2697])}

In [16]:
!nvidia-smi

Sun Jul  4 12:59:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:

limit_idx = 100
for key in train_model_input:
  train_model_input[key] = train_model_input[key][:limit_idx]


In [18]:
train_model_input['hist_movie_id'][0]

array([1171, 1032, 1770, 3108, 3229,  528,  899, 2094,  861, 1141,  654,
       2722,  241, 1101,  695,  254, 2204,  849, 1109, 1158, 1117, 2937,
        288,  882,  859, 3133, 1114, 1156, 3652, 1769, 3509, 1033, 2977,
       1203, 1013, 1185,  594, 1165, 3495, 1831, 1071,  866, 1251, 3085,
        880, 2447, 2296, 1128, 1762, 1163], dtype=int32)

In [19]:
idiot = ",".join(list(map(lambda i: str(i), train_model_input['hist_movie_id'][0])))

In [20]:
tmp_hist_movie_id = []
for i in range(train_model_input['hist_movie_id'].shape[0]):
  s_hist_movie_id = ",".join(list(map(lambda i: str(i), train_model_input['hist_movie_id'][i])))
  tmp_hist_movie_id.append(s_hist_movie_id)


In [21]:
modified_train_model_input = dict(train_model_input)
modified_train_model_input['hist_movie_id'] = tmp_hist_movie_id

modified_train_model_input['label'] = [1] * len(modified_train_model_input['hist_movie_id'])

df_train = pd.DataFrame.from_dict(modified_train_model_input)

In [22]:
df_train.head()

,user_id,movie_id,hist_movie_id,hist_len,gender,age,occupation,zip,label
0,1140,1159,"1171,1032,1770,3108,3229,528,899,2094,861,1141...",91,1,5,3,2295,1
1,4771,2721,"1048,1937,964,853,1776,1208,2132,2887,989,1501...",287,1,3,5,227,1
2,4028,155,"93,1598,2219,1273,3597,421,3174,148,2175,168,1...",277,2,3,5,106,1
3,5534,880,"1752,859,1109,254,1171,1107,1108,2898,2840,267...",31,2,5,17,1077,1
4,5208,107,"2375,159,1595,2402,2776,2559,2471,2635,2498,29...",48,2,3,5,1939,1


In [23]:
df_train.to_csv("./part_data", sep='\t', index=False)

In [24]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
456790,6040,803,4,956703932,"Godfather, The (1972)",Action|Crime|Drama,2,3,7,467
456672,6040,580,5,956703954,"Silence of the Lambs, The (1991)",Drama|Thriller,2,3,7,467
456732,6040,2192,4,956703954,Babe: Pig in the City (1998),Children's|Comedy,2,3,7,467
456641,6040,1782,4,956703977,Rain Man (1988),Drama,2,3,7,467
456842,6040,1840,5,956703977,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama,2,3,7,467


In [25]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers

user_sparse_features = ["user_id", "gender", "age", "occupation", "zip", ]
user_var_len_features = ['hist_movie_id']
user_dense_features = []
def build_feature_columns(sparse_features, dense_features, var_len_features):
  #define feature colunmns here
  normal_feature_columns = []
  var_len_feature_columns = []
  all_inputs = {}

  # indicator_columns

  for col_name in sparse_features:
    categorical_column = tf.feature_column.categorical_column_with_identity(col_name, 20000, default_value=0) #fixed the value
    indicato_embedding = feature_column.embedding_column(categorical_column, dimension=16)  #fix first, later change to configurable
    normal_feature_columns.append(indicato_embedding)
    categorical_input_col = tf.keras.Input(shape=(1,), name=col_name)
    all_inputs[col_name] = categorical_input_col


  # numeric cols
  
  for col_name in dense_features:
    normal_feature_columns.append(feature_column.numeric_column(col_name))
    numeric_input_col = tf.keras.Input(shape=(1,), name=col_name)
    all_inputs[col_name] = numeric_input_col

  
  # var_len_features = []
  for col_name in var_len_features:
    seq_categorical_column = tf.feature_column.sequence_categorical_column_with_identity(
        col_name, num_buckets=20000, default_value=0) #hist_movie_id is composed of movie_id
    seq_categorical_embedding = tf.feature_column.embedding_column(seq_categorical_column, dimension=16)
    var_len_feature_columns.append(seq_categorical_embedding)
    var_len_input_col = tf.keras.Input(shape=(50,), name=col_name)  #fixed first
    all_inputs[col_name] = var_len_input_col
  return normal_feature_columns, var_len_feature_columns, all_inputs

user_normal_feature_columns, user_var_len_feature_columns, user_all_inputs = build_feature_columns(user_sparse_features, user_dense_features, user_var_len_features)

item_sparse_features = ["movie_id"]
item_var_len_features = []
item_dense_features = []
item_normal_feature_columns, item_var_len_feature_columns, item_all_inputs = build_feature_columns(item_sparse_features, item_dense_features, item_var_len_features)


In [26]:

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())
def seq_demo(feature_column):
  feature_layer = tf.keras.experimental.SequenceFeatures(feature_column)
  sequence_input, sequence_length = feature_layer(example_batch)
  sequence_length_mask = tf.sequence_mask(sequence_length)
  print(sequence_input.numpy())

In [27]:

train_ds = tf.data.experimental.make_csv_dataset(
    './part_data', field_delim='\t',
    batch_size=4, # Artificially small to make examples easier to show.
    label_name='label',
    num_epochs=1,
    ignore_errors=True,)


In [28]:
@tf.function 
def string_to_tensor(s):
  st = tf.strings.split(s, sep=',')
  return tf.strings.to_number(st,out_type=tf.dtypes.int32).to_tensor()


In [29]:
class DNN(tf.keras.Model):
  def __init__(self):
    super().__init__()
    
    self.dense1 = tf.keras.layers.Dense(32, activation='relu')
    self.dense2 = tf.keras.layers.Dense(16, activation=None)

  @tf.function
  def __call__(self, inputs, training=False):
    x = self.dense1(inputs)
    res = self.dense2(x)
    return res

class Basic(tf.keras.Model):
  def __init__(self, user_normal_feature_columns, user_var_len_feature_columns, user_var_len_feature_keys,
               item_normal_feature_columns, item_var_len_feature_columns, item_var_len_feature_keys):
    super().__init__()
    
    self.preprocess_var_len = string_to_tensor
    self.flatten_layer = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(32, activation='relu')
    self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')
    self.user_normal_feature_columns = user_normal_feature_columns
    self.user_var_len_feature_columns = user_var_len_feature_columns
    self.user_var_len_feature_keys = user_var_len_feature_keys
    self.user_feature_layer = tf.keras.layers.DenseFeatures(self.user_normal_feature_columns)
    if user_var_len_feature_columns:
      self.user_val_len_feature_layer = tf.keras.experimental.SequenceFeatures(self.user_var_len_feature_columns)
    else:
      self.user_val_len_feature_layer = None
    self.item_normal_feature_columns = item_normal_feature_columns
    self.item_var_len_feature_columns = item_var_len_feature_columns
    self.item_var_len_feature_keys = item_var_len_feature_keys
    self.item_feature_layer = tf.keras.layers.DenseFeatures(self.item_normal_feature_columns)
    if item_var_len_feature_columns:
      self.item_val_len_feature_layer = tf.keras.experimental.SequenceFeatures(self.item_var_len_feature_columns)
    else:
      self.item_val_len_feature_layer = None
    self.user_model = DNN()
    self.item_model = DNN()

  @tf.function
  def __call__(self, inputs, training=False):
    user_all_features = self.process_features(inputs, feature_layer=self.user_feature_layer, val_len_feature_layer=self.user_val_len_feature_layer, 
                                              var_len_features=self.user_var_len_feature_keys,)
    user_embedding = self.get_embedding(features=user_all_features, model=self.user_model)
    item_all_features = self.process_features(inputs, feature_layer=self.item_feature_layer, val_len_feature_layer=self.item_val_len_feature_layer, 
                                              var_len_features=self.item_var_len_feature_keys,)
    item_embedding = self.get_embedding(features=item_all_features, model=self.item_model)
    user_embedding = user_embedding / tf.expand_dims(tf.sqrt(tf.reduce_sum(user_embedding ** 2, axis=-1)), axis=-1)
    item_embedding = item_embedding / tf.expand_dims(tf.sqrt(tf.reduce_sum(item_embedding ** 2, axis=-1)), axis=-1)
    res = tf.reduce_sum(user_embedding * item_embedding, axis=-1)
    # return res
    predicted_ctr = tf.math.sigmoid(res)
    return predicted_ctr
  
  def process_features(self, inputs, feature_layer, val_len_feature_layer, var_len_features):
    normal_features = feature_layer(inputs)
    var_len_inputs = {}
    for key in var_len_features:
      var_len_inputs[key] = string_to_tensor(inputs[key])
    if val_len_feature_layer:
      sequence_features, sequence_length = val_len_feature_layer(var_len_inputs)
      sequence_length_mask = tf.sequence_mask(sequence_length) 
      all_features = tf.keras.layers.concatenate([self.flatten_layer(normal_features), tf.math.reduce_mean(sequence_features, axis=1)])
    else:
      all_features = self.flatten_layer(normal_features)
    return all_features

  def get_embedding(self, features, model):
    return model(features)

  # def get_user_embedding(self, user_features):
  #   return self.user_model(user_features)
  
  # def get_item_embedding(self, item_features):
  #   return self.item_model(item_features)


In [31]:
model = Basic(user_normal_feature_columns=user_normal_feature_columns, user_var_len_feature_columns=user_var_len_feature_columns, user_var_len_feature_keys=user_var_len_features,
               item_normal_feature_columns=item_normal_feature_columns, item_var_len_feature_columns=item_var_len_feature_columns, item_var_len_feature_keys=item_var_len_features)    
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              )
              # run_eagerly=True)
              # metrics=["accuracy"])
              # metrics=["accuracy"], run_eagerly=True)
model.fit(train_ds, epochs=10)

Epoch 1/10
25/25 [==============================] - 3s 18ms/step - loss: 0.5885
Epoch 2/10
25/25 [==============================] - 0s 17ms/step - loss: 0.4201
Epoch 3/10
25/25 [==============================] - 0s 17ms/step - loss: 0.3715
Epoch 4/10
25/25 [==============================] - 0s 17ms/step - loss: 0.3524
Epoch 5/10
25/25 [==============================] - 0s 15ms/step - loss: 0.3424
Epoch 6/10
25/25 [==============================] - 0s 17ms/step - loss: 0.3358
Epoch 7/10
25/25 [==============================] - 0s 17ms/step - loss: 0.3314
Epoch 8/10
25/25 [==============================] - 0s 16ms/step - loss: 0.3282
Epoch 9/10
25/25 [==============================] - 0s 17ms/step - loss: 0.3258
Epoch 10/10
25/25 [==============================] - 0s 16ms/step - loss: 0.3239
